In [1]:
%pip install -i https://test.pypi.org/simple/ enet-seifeddine-dridi --force-reinstall

Looking in indexes: https://test.pypi.org/simple/
  Using cached https://test-files.pythonhosted.org/packages/fa/85/aa108f2ec2d93a2696e74ce06bb8ea6af0be77ed868459f4248cd81d0d4a/enet_seifeddine_dridi-0.0.19-py3-none-any.whl.metadata (3.7 kB)
Using cached https://test-files.pythonhosted.org/packages/fa/85/aa108f2ec2d93a2696e74ce06bb8ea6af0be77ed868459f4248cd81d0d4a/enet_seifeddine_dridi-0.0.19-py3-none-any.whl (16 kB)
  Attempting uninstall: enet-seifeddine-dridi
    Found existing installation: enet_seifeddine_dridi 0.0.19
    Uninstalling enet_seifeddine_dridi-0.0.19:
      Successfully uninstalled enet_seifeddine_dridi-0.0.19
Note: you may need to restart the kernel to use updated packages.


Setup the project configuration

In [2]:
from time import time

import torch
from tqdm import tqdm
from lanenet.enet.config import EnetConfig
from lanenet.enet.model_utils import load_model, eval_model, compute_loss

In [4]:
# pretrained_model_path = None
pretrained_model_path = '../pretrained_model/enet_model_829.pt'
config = EnetConfig(pretrained_model_path=pretrained_model_path, train_full_model=True, max_epoch=1000, dataset_root_folder='../datasets/cityscapes/data_unzipped')
device = config.device
torch.multiprocessing.set_start_method('spawn')
torch.set_flush_denormal(True)
model, optimizer, train_dataset, test_dataset, last_run_epoch,  best_eval_loss = load_model(config, device)
train_dataset_iter = iter(train_dataset)
test_dataset_iter = iter(test_dataset)

last_checkpoint_saving_time = time()
saving_period = 2 * 60  # 2 minutes

Checkpoint file ../pretrained_model/enet_model_829.pt successfully loaded


In [5]:
bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, ''{rate_noinv_fmt}{postfix}]'
progress_bar = tqdm(range(last_run_epoch, config.max_epoch), bar_format=bar_format)
for epoch in progress_bar:
    try:
        in_tensor, target = next(train_dataset_iter)
    except StopIteration:
        # Iterator is exhausted
        train_dataset_iter = iter(train_dataset)
        in_tensor, target = next(train_dataset_iter)
    in_tensor = in_tensor.to(device)
    target = target.to(device)
    logits = model(in_tensor)
    loss = compute_loss(logits, target, config.custom_weight_scaling_const, config.scaling_props_range)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    progress_bar.set_description(f"Epoch [{epoch}/{config.max_epoch}]")
    progress_bar.set_postfix(loss=loss.item())
    if time() - last_checkpoint_saving_time >= saving_period  or epoch == config.max_epoch - 1:
        last_checkpoint_saving_time = time()
        try:
            eval_loss = eval_model(model, test_dataset_iter, config.custom_weight_scaling_const,
                                   config.scaling_props_range, device)
        except StopIteration:
            # Iterator is exhausted
            test_dataset_iter = iter(test_dataset)
            eval_loss = eval_model(model, test_dataset_iter, config.custom_weight_scaling_const,
                                   config.scaling_props_range, device)
        if eval_loss < best_eval_loss:
            config.save_checkpoint(model, optimizer, loss, epoch, '../')
            best_eval_loss = eval_loss

  0%|                                                                                                                                                                                                           | 0/171 [00:00<?, ?it/s]/Users/seifeddine/Programming/Personal/lane_detection/venv/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1041.)
  return F.conv2d(input, weight, bias, self.stride,
Epoch [840/1000]:   7%|███████████▌                                                                                                                                                         | 12/171 [00:28<06:17,  0.42it/s, loss=1.38]


KeyboardInterrupt: 